**Importing Library**

In [1]:
import autokeras as ak
from tensorflow import keras
import tensorflow as tf
import cv2
import glob
import numpy as np
import os
import pandas as pd

**Preparing data**

In [2]:
X_train = []
files = glob.glob ("dataset/train_mag/*.jpg")
for myFile in files:
    #print(myFile)
    image = cv2.imread (myFile)
    X_train.append (image)
X_train = np.array(X_train)
print('X_train shape:', np.array(X_train).shape)

X_test = []
files = glob.glob ("dataset/test_mag/*.jpg")
for myFile in files:
    #print(myFile)
    image = cv2.imread (myFile)
    X_test.append (image)
X_test = np.array(X_test)
print('X_test shape:', np.array(X_test).shape)

df = pd.read_csv("dataset/train_mag.csv")
Y_train = df.values[:,1]
print(Y_train.shape)

df1 = pd.read_csv("dataset/test_mag.csv")
Y_test = df1.values[:,1]
print(Y_test.shape)

X_train shape: (2518, 128, 128, 3)
X_test shape: (163, 128, 128, 3)
(2518,)
(163,)


In [3]:
# Initialize the image classifier.
clf = ak.ImageClassifier(
    overwrite=True,
    max_trials=10)
# Feed the image classifier with training data.
clf.fit(X_train, Y_train, epochs=10)

# Predict with the best model.
predicted_y = clf.predict(X_test)
print(predicted_y)

# Evaluate the best model with testing data.
print(clf.evaluate(X_test, Y_test))

Trial 6 Complete [01h 18m 10s]
val_loss: 3.662950277328491

Best val_loss So Far: 1.2244194746017456
Total elapsed time: 05h 47m 50s

Search: Running Trial #7

Hyperparameter      |Value     |Best Value So Far   
image_block_1/normalize|True      |True                
image_block_1/augment|True      |True                
image_block_1/block_type|resnet    |resnet              
image_block_1/res_net_block_1/pretrained|True      |True                
image_block_1/res_net_block_1/version|resnet50  |resnet50            
image_block_1/res_net_block_1/imagenet_size|True      |True                
classification_head_1/spatial_reduction_1/reduction_type|global_avg|global_avg          
classification_head_1/dropout|0         |0                   
optimizer           |adam      |adam                
learning_rate       |1e-05     |1e-05               
image_block_1/image_augmentation_1/translation_factor|0.1       |0.1                 
image_block_1/image_augmentation_1/horizontal_flip|True   

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
model = clf.export_model()

In [5]:
#model
model.save("model_autokeras_magtrain.h5")

In [6]:
model=keras.models.load_model("model_autokeras_magtrain.h5")

In [6]:
model.summary(100)

Model: "functional_1"
____________________________________________________________________________________________________
Layer (type)                                 Output Shape                            Param #        
input_1 (InputLayer)                         [(None, 128, 128, 3)]                   0              
____________________________________________________________________________________________________
normalization (Normalization)                (None, 128, 128, 3)                     7              
____________________________________________________________________________________________________
random_translation (RandomTranslation)       (None, 128, 128, 3)                     0              
____________________________________________________________________________________________________
random_flip (RandomFlip)                     (None, 128, 128, 3)                     0              
_____________________________________________________________________

In [8]:
try:
    model.save("model_autokeras", save_format="tf")
except:
    model.save("model_autokeras.h5")


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_autokeras/assets


In [11]:
from tensorflow.keras.models import load_model

loaded_model = load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)

predicted_y = loaded_model.predict(tf.expand_dims(X_test, -1))
print(predicted_y)

[[3.5592262e-02 4.8086382e-03 5.8584084e-04 3.3004687e-04 9.3967873e-01
  1.9004505e-02]
 [1.7515214e-02 2.1928538e-02 1.4621833e-02 1.3828835e-03 8.7197465e-01
  7.2576858e-02]
 [6.9361806e-02 9.3843965e-03 1.6748859e-02 1.5692002e-04 8.5828120e-01
  4.6066787e-02]
 ...
 [2.1882723e-03 7.1585015e-03 1.7117480e-02 1.2428820e-04 9.3521976e-01
  3.8191747e-02]
 [1.1451409e-02 3.2205187e-02 4.2341961e-04 6.1835221e-05 9.4062209e-01
  1.5236021e-02]
 [9.5045110e-03 1.7608872e-02 4.4696368e-03 9.8745222e-04 9.3538880e-01
  3.2040749e-02]]
